Let's download the data from INSEE.

In [311]:
import pandas as pd


url = "https://www.insee.fr/fr/statistiques/fichier/4648335/TF106.csv"

df = pd.read_csv(url, sep=";")

In [312]:
df = df.drop(df[df['DECUC'] == 'TOT'].index)

In [313]:
# filtering values with exactly 4 characters, corresponding to ECOICOP classes
df = df.loc[df['NOMENCLATURE'].str.len() == 4]

In [314]:
# renaming the "NOMENCLATURE" column to "FCOICOP"
df = df.rename(columns={'NOMENCLATURE': 'FCOICOP'})

In [315]:
# filtering out rows where the "FCOIC" value starts with "13"
df = df.loc[~df['FCOICOP'].str.startswith('13')]

In [316]:
# loading the second csv file with the "FCOICOP" column
df2 = pd.read_csv('Nomenclature_classes_to_special_aggregates.csv')

In [317]:
# merging the two dataframes on the "FCOICOP" column
merged_df = pd.merge(df, df2, on='FCOICOP')

In [318]:
# calculating the sum of "CONSO" for each unique value in "DECUC" column
sum_by_decuc = merged_df.groupby('DECUC')['CONSO'].sum()

In [319]:
# adding a new column "PROP" with the percentage of each row relative to the sum for each unique value in "DECUC" column
merged_df['PROP'] = merged_df['CONSO'] / merged_df['DECUC'].map(sum_by_decuc)

In [320]:
%matplotlib agg
import matplotlib.pyplot as plt

# group the merged dataframe by "AGGREGATE" and "DECUC" and sum the values in the "PROP" column for each group
grouped_df = merged_df.groupby(['AGGREGATE', 'DECUC'])['PROP'].sum().reset_index()


# create a new column "DECUC_SORTED" with values sorted in a specific order
decuc_order = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
grouped_df['DECUC_SORTED'] = pd.Categorical(grouped_df['DECUC'], categories=decuc_order, ordered=True)

# pivot the dataframe to create a table where each row represents a DECUC, and each column represents an AGGREGATE with the values in the "PROP" column
pivoted_df = grouped_df.pivot(index='DECUC_SORTED', columns='AGGREGATE', values='PROP').fillna(0)


# calculate the cumulative sum of the values in the pivoted dataframe
cumulative_df = pivoted_df.cumsum(axis=1)



# create a stacked bar chart with a bar for each DECUC and bars of colors with sizes proportional to PROP for each AGGREGATE
fig, ax = plt.subplots(figsize=(9,15))

for i, (colname, color) in enumerate(zip(pivoted_df.columns, ['blue', 'green', 'red', 'purple'])):
    ax.bar(pivoted_df.index, pivoted_df[colname], bottom=cumulative_df.iloc[:, i-1] if i>0 else None, width=0.9, color=color, alpha=0.8, label=colname)
    for j, val in enumerate(pivoted_df[colname]):
        if val > 0:
            ax.text(j, cumulative_df.iloc[j, i-1] if i>0 else 0, f"{round(val*100)}%", ha='center', va='bottom', color='white', fontweight='bold', fontsize=20)

# change the legend labels using a dictionary
legend_labels = {'FOOD': 'Food', 'IGD': 'Industrial goods', 'SERV': 'Services', 'NRG': 'Energy'}
handles, labels = ax.get_legend_handles_labels()
new_labels = [legend_labels[label] if label in legend_labels else label for label in labels]
ax.legend(handles, new_labels, title="Components", fontsize='large')


ax.set_xlabel("Deciles")
ax.set_ylabel("Proportion of consumption by component")

ax.tick_params(axis='both', labelsize='large')
ax.set_xlabel("Deciles", fontsize='large')
ax.set_ylabel("Proportion of consumption by component", fontsize='large')

# add a "D" prefix to the tick labels of the x-axis
ticks = ['D' + tick for tick in pivoted_df.index]
plt.xticks(range(len(ticks)), ticks)

plt.title("Proportion of annual household expenditures in France by deciles in 2017", fontsize=20)
plt.show()

plt.rcParams['font.family'] = 'cmr10'
plt.savefig('saved_figs/figure4.png', dpi=500, bbox_inches='tight')
plt.show()

C:\Users\maxen\AppData\Local\Temp\ipykernel_19124\2297195716.py:49: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\maxen\AppData\Local\Temp\ipykernel_19124\2297195716.py:53: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
